In [ ]:
import os
import numpy as np
import sys
print(sys.executable)
import sys
sys.path.insert(1, '../src/')
from CNN_classifier_model import CNNClassifier, train_model, evaluate_model
from config import raw_data_path, univariate_data_path, processed_data_path
from preprocessing_modules import create_time_windows_with_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import wandb
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from FCMAE_model import FCMAE
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")


In [42]:
# import wandb

# wandb.init(project="XGBoost for time series classification", config={
#     "model": "XGBoost",
#     "input_length": 12000,
#     "batch_size": batch_size,
#     "lr": 1e-3,
#     "epochs": 10,
#     "dropout": 0.2,
#     "kernel_size": 7,
#     "channels": [32, 64],
# })


In [ ]:
train_dir = os.path.join(univariate_data_path, "target_univariate_train.npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
test_dir = os.path.join(univariate_data_path, "target_univariate_test.npy")
test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
print(len(test_data))
print(train_data[0])

train_windows = create_time_windows_with_labels(train_data)
test_windows = create_time_windows_with_labels(test_data)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter

# Example dataset with windows and labels
train_windows, train_labels = create_time_windows_with_labels(train_data)
test_windows, test_labels = create_time_windows_with_labels(test_data)

# Count label distribution
train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)

# Print the counts
print("Train label distribution:")
print(f"  Term (0): {train_label_counts[0]}")
print(f"  Preterm (1): {train_label_counts[1]}")

print("\nTest label distribution:")
print(f"  Term (0): {test_label_counts[0]}")
print(f"  Preterm (1): {test_label_counts[1]}")
print(train_windows.shape, train_labels.shape)

In [ ]:
path = os.path.join(models_path, 'FCMAE_best_multiple_encoder_cpu.pth')

# Load the full FCMAE model
fcmae = FCMAE(in_channels=1, base_dim=128, num_blocks=5, kernel_size=9)


# Load the saved weights into the full model
state_dict = torch.load(path)
fcmae.load_state_dict(state_dict)

# Extract only the encoder from the FCMAE model
encoder = fcmae.encoder
# Set the encoder to evaluation mode and freeze the encoder parameters
encoder.eval()

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# No need to reshape, already correct shape
X_train, y_train = train_windows, train_labels
X_test, y_test = test_windows, test_labels


model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=train_label_counts[0] / train_label_counts[1],
    n_estimators=500,  # number of boosting rounds (like epochs), 
    reg_alpha=1.0,
    reg_lambda=1.0,
    max_depth=3,
    learning_rate=0.01
)

# Provide validation set to track loss
eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

# Retrieve evaluation results
results = model.evals_result()

# Plot
plt.figure(figsize=(10, 5))
plt.plot(results['validation_0']['logloss'], label='Train Loss')
plt.plot(results['validation_1']['logloss'], label='Test Loss')
plt.xlabel('Boosting Round')
plt.ylabel('Log Loss')
plt.title('XGBoost Train/Test Loss per Boosting Round')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming train_windows and train_labels are already loaded
# Split the training data into 80% training and 20% validation
train_windows_split, val_windows_split, train_labels_split, val_labels_split = train_test_split(
    train_windows, train_labels, test_size=0.2, random_state=42
)

# Print the split sizes for verification
print(f"Train set size: {train_windows_split.shape[0]} samples")
print(f"Validation set size: {val_windows_split.shape[0]} samples")
print(train_windows_split.shape)
print(train_labels_split.shape)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# No need to reshape, already correct shape
X_train, y_train = train_windows_split, train_labels_split
X_test, y_test = val_windows_split, val_labels_split


model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=train_label_counts[0] / train_label_counts[1],
    n_estimators=500,  # number of boosting rounds (like epochs), 
    reg_alpha=1.0,
    reg_lambda=1.0,
    max_depth=3,
    learning_rate=0.01
)

# Provide validation set to track loss
eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

# Retrieve evaluation results
results = model.evals_result()

# Plot
plt.figure(figsize=(10, 5))
plt.plot(results['validation_0']['logloss'], label='Train Loss')
plt.plot(results['validation_1']['logloss'], label='Test Loss')
plt.xlabel('Boosting Round')
plt.ylabel('Log Loss')
plt.title('XGBoost Train/Test Loss per Boosting Round')
plt.legend()
plt.grid(True)
plt.show()
